<a href="https://colab.research.google.com/github/YuTaNCCU/201902_ANN_Metaheuristic/blob/master/ES/ES_ANN_0413.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Package

In [0]:
import random
from string import ascii_lowercase
from copy import deepcopy
from abc import ABCMeta, abstractmethod
from copy import deepcopy
from collections import deque
from numpy import argmax
from keras import backend as K
from keras.models import Sequential 
import numpy as np


class ES:
    """
    Conducts tabu search
    """
    __metaclass__ = ABCMeta

    #default hyper parameters
    InitialSigma = None
    ParentsSize = None
    ChildSize = None
    tao = None
    
    #for input/output
    KerasModels = None
    WeightsStrucure = None   
    weights = None
    
    #for record
    cur_steps = 1
    best_weight = None
    best_score = None
    
    def __init__(self, KerasModels, InitialSigma = 0.1, ParentsSize = 15, ChildSize = 100, tao = 0.5):
        """
        :param KerasModels: a Keras model, like keras.engine.sequential.Sequential
        :param weights: initial weights, should be a Keras model weight
        :param max_steps: maximum number of steps to run algorithm for
        """
        self.KerasModels = KerasModels
 
        if all(isinstance(x, float) for x in [InitialSigma, tao]) and all(x > 0 for x in [InitialSigma, tao]):
            self.InitialSigma = InitialSigma
            self.tao = tao
        else:
            raise TypeError('InitialSigma & tao must be a positive float')
            
        if all(isinstance(x, int) for x in [ParentsSize, ChildSize]) and all(x > 0 for x in [ParentsSize, ChildSize]):
            self.ParentsSize = ParentsSize
            self.ChildSize = ChildSize
        else:
            raise TypeError('ParentsSize, ChildSize & max_steps must be a positive integer')

    def __str__(self): 
        return ('ES: \n' +
                'CURRENT STEPS: %d \n' +
                'BEST MEMBER: \n %s ......\n' +
                'BEST SCORE%f \n') % \
               (self.cur_steps, str(self.best_weight)[:100], self.best_score)

    def __repr__(self):
        return self.__str__() 
    
    def _FlattenWeights(self, weights):
        """
        flatten weights
        
        param weights: keras神經網路的權重格式:nparray包在list中
        return WeightsStrucure : 神經網路各層的權重shape包在list中，unflatten時會用到
        return FlattenedWeights : 一維list包含所有的權重
        """
        WeightsStrucure = []
        FlattenedWeights = []
        for i_layer in weights:
            WeightsStrucure.append(i_layer.shape)
            if len(i_layer.shape) == 1 :# 該層權重的shape為一維 e.g. (15,)      
                FlattenedWeights.extend(i_layer)
            else :# 該層權重的shape為二維 e.g. (30, 15)  
                for i_links in i_layer:
                    FlattenedWeights.extend(i_links)
        return WeightsStrucure, FlattenedWeights

    def _UnflattenWeights(self, WeightsStrucure, ModifiedWeights):
        """
        Unflatten(回復成原本的結構) weights  
        
        param WeightsStrucure : 神經網路各層的權重shape包在list中
        param ModifiedWeights : 一維list包含所有meteHeuristic修改過的權重
        return: keras神經網路的權重格式:nparray包在list中
        """
        UnflattenWeights = []
        i_index = 0 
        for i_layer in WeightsStrucure:
            if len(i_layer) == 1 : # 該層權重的shape為一維 e.g. (15,)      
                TempList = ModifiedWeights[i_index:(i_index + i_layer[0])]
                TempList = np.asarray(TempList)
                i_index = i_index + i_layer[0]
            else : # 該層權重的shape為二維 e.g. (30, 15)  
                TempList = ModifiedWeights[i_index:(i_index + (i_layer[0]*i_layer[1]))]
                TempList = np.reshape(TempList, i_layer )
                i_index = i_index + (i_layer[0]*i_layer[1])
            UnflattenWeights.append(TempList)
        return UnflattenWeights   
    
    def _best(self, Population_Child_score):
        """
        Finds the best member of a neighborhood
        :param Population_Child_score: a np array
        :return: the indtex of N best member, N = ParentsSize
        """
        return np.array( Population_Child_score ).argsort()[::-1][:self.ParentsSize]
    
    def _Recombination(self, Population_Parents_Weights, Population_Parents_Sigma, rows): #GenerateParents
        """
        Generate New Parents Polulation
        """
        Population_Weights_Recombination = np.zeros(shape = (rows, Population_Parents_Weights.shape[1]))
        Population_Sigma_Recombination = np.zeros(shape = (rows, Population_Parents_Weights.shape[1]))
        for index_row, _ in enumerate( Population_Weights_Recombination ):
            """
            可能可以平行計算
            """
            TwoRowschoiced = np.random.choice(Population_Parents_Weights.shape[0], size=2, replace=False,)
            Parent1Mask = np.random.randint(2, size=Population_Parents_Weights.shape[1])
            Parent2Mask = np.full(shape = Population_Parents_Weights.shape[1], fill_value = 1 )  - Parent1Mask
            
            Population_Weights_Recombination[index_row,:] = (Population_Parents_Weights[TwoRowschoiced] * [Parent1Mask, Parent2Mask]).sum(axis=0)
            Population_Sigma_Recombination[index_row,:] = Population_Parents_Sigma[TwoRowschoiced].mean(axis=0)
        return Population_Weights_Recombination, Population_Sigma_Recombination
    
    def _score(self, ModifiedWeights):
        
        """
        Returns objective function value of a state

        :param state: a state
        :return: objective function value of state
        """
        
        UnflattenedWeights = self._UnflattenWeights(WeightsStrucure = self.WeightsStrucure, ModifiedWeights = ModifiedWeights)
        self.KerasModels.set_weights(UnflattenedWeights)
        test_on_batch = self.KerasModels.test_on_batch(X_train, y_train, sample_weight=None) # return ['loss', 'acc']
        return test_on_batch[1]

    def run(self, weights, max_steps=5, verbose=10):
        """
        Conducts ES
        :param weights: 
        :param max_steps: 
        :param verbose: int which indicates how many iter to show score
        :return: Keras Models, best state and objective function value of best state
        """
        
        if isinstance(weights, list)  :
          
            self.WeightsStrucure, self.weights = self._FlattenWeights(weights)
            self.best_weight = self.weights
            self.best_score = self._score(self.best_weight)
        else:
            raise TypeError('initial_state must be a list') 
            
        self.max_steps = max_steps
        
        #Step1 initial             
        Population_Parents_Weights = np.array([self.weights, self.weights])         
        Population_Parents_Sigma = np.full(shape = (self.ParentsSize, len(self.weights)), fill_value = self.InitialSigma ) 
        Population_Parents_Weights, _ = self._Recombination(Population_Parents_Weights, Population_Parents_Sigma, rows = self.ParentsSize )
        self.cur_steps = 1
        while True:   
            #Step2 Child
            ##Discrete Recombination
            Population_Child_Weights, Population_Child_Sigma = self._Recombination(Population_Parents_Weights, Population_Parents_Sigma, rows = self.ChildSize )
            ##mutation1
            RamdonNormalValue = np.random.normal(0, 1, 1)
            RamdonNormalValueDifferent = np.random.normal(0, 1, Population_Child_Sigma.shape)
            Population_Child_Sigma = np.exp( (1-self.tao)*RamdonNormalValue + self.tao*RamdonNormalValueDifferent )
            ##mutation2
            Population_Child_Weights = Population_Child_Weights + np.random.normal(0, Population_Child_Sigma, Population_Child_Sigma.shape)
            
            #step3 Evaluation
            Population_Child_score = []
            for i_Child in Population_Child_Weights :
                """
                可能可以平行計算
                """
                Population_Child_score.append( self._score(i_Child) )
            BestNIndex = self._best(Population_Child_score)
            Population_Parents_Weights = Population_Child_Weights[BestNIndex,:]
            Population_Parents_Sigma = Population_Child_Sigma[BestNIndex,:]
            
            #更新best
            best_weight_This_Iter =  Population_Child_Weights[BestNIndex,:][0]
            best_score_This_Iter = self._score(Population_Child_Weights[BestNIndex,:][0])
            if best_score_This_Iter > self.best_score:
                self.best_weight =  Population_Child_Weights[BestNIndex,:][0]
                self.best_score = self._score(Population_Child_Weights[BestNIndex,:][0])
           
            #step4 check stop criteria
            if self.cur_steps > max_steps:
                print( 'Stop: Reach max_steps' )
                break
            
            #print process 
            if ((self.cur_steps ) % verbose == 0) and verbose:
               print(self)
               
            self.cur_steps = self.cur_steps + 1
        return self._UnflattenWeights(WeightsStrucure = self.WeightsStrucure, ModifiedWeights = self.best_weight), self.best_score 


# Load Data

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer

#資料集是以dictionary的形式存在
cancer = load_breast_cancer()
df_feat = pd.DataFrame(cancer['data'],columns=cancer['feature_names'])

X = df_feat.iloc[:, ].values
y = cancer['target']

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder
labelencoder_X_1 = LabelEncoder()
y = labelencoder_X_1.fit_transform(y)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

#Feature Scaling
"""from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)"""

#X_train.shape,X_test.shape,y_train.shape,y_test.shape

'from sklearn.preprocessing import StandardScaler\nsc = StandardScaler()\nX_train = sc.fit_transform(X_train)\nX_test = sc.transform(X_test)'

# Model Compile

In [8]:
from keras import backend as K
from keras.layers import Dense
from keras.models import Sequential, Model as keras_models_Model

model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(30,)))
#model.add(Dense(3, activation='relu'))
model.add(Dense(3, activation='relu', name = 'IntermediateLayer'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                310       
_________________________________________________________________
IntermediateLayer (Dense)    (None, 3)                 33        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 4         
Total params: 347
Trainable params: 347
Non-trainable params: 0
_________________________________________________________________


# Initialize

In [4]:

weights = model.get_weights() 
#a=_FlattenWeights(weights)
MyES = ES(model, InitialSigma = 0.1, ParentsSize = 15, ChildSize = 100, tao = 0.5)   
ES_Optimized_Weights, ES_Optimized_ObjVal  = MyES.run(weights, max_steps=5, verbose = 1)

ES: 
CURRENT STEPS: 1 
BEST MEMBER: 
 [-3.89431480e-01  2.40068441e-01 -2.33282341e+00  5.61662770e+00
  2.00720283e+00 -3.64790010e+00 -3 ......
BEST SCORE0.861538 

ES: 
CURRENT STEPS: 2 
BEST MEMBER: 
 [-3.89431480e-01  2.40068441e-01 -2.33282341e+00  5.61662770e+00
  2.00720283e+00 -3.64790010e+00 -3 ......
BEST SCORE0.861538 

ES: 
CURRENT STEPS: 3 
BEST MEMBER: 
 [-3.89431480e-01  2.40068441e-01 -2.33282341e+00  5.61662770e+00
  2.00720283e+00 -3.64790010e+00 -3 ......
BEST SCORE0.861538 

ES: 
CURRENT STEPS: 4 
BEST MEMBER: 
 [-3.89431480e-01  2.40068441e-01 -2.33282341e+00  5.61662770e+00
  2.00720283e+00 -3.64790010e+00 -3 ......
BEST SCORE0.861538 

ES: 
CURRENT STEPS: 5 
BEST MEMBER: 
 [-3.81060127e+00  1.27389825e+00  2.11697893e+00 -5.69899309e+00
  5.23015233e+00 -3.48286970e+00  5 ......
BEST SCORE0.890110 

Stop: Reach max_steps


# Main

In [5]:
# gradient-based optimize
model.set_weights(ES_Optimized_Weights)
model.fit(X_train, y_train, epochs=5, batch_size=32)
weights = model.get_weights() 

# ES
ES_Optimized_Weights, ES_Optimized_ObjVal  = MyES.run(weights, max_steps=10, verbose = 1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
455/455 [==============================] - 0s 578us/step - loss: 0.6157 - acc: 0.8923
Epoch 2/5
455/455 [==============================] - 0s 48us/step - loss: 0.6138 - acc: 0.8923
Epoch 3/5
455/455 [==============================] - 0s 46us/step - loss: 0.6129 - acc: 0.8923
Epoch 4/5
455/455 [==============================] - 0s 46us/step - loss: 0.6120 - acc: 0.8923
Epoch 5/5
455/455 [==============================] - 0s 45us/step - loss: 0.6110 - acc: 0.8923
ES: 
CURRENT STEPS: 1 
BEST MEMBER: 
 [-3.8050463, 1.2683436, 2.116979, -5.70455, 5.2357087, -3.488425, 5.2477236, -2.6602743, 0.5050497,  ......
BEST SCORE0.892308 

ES: 
CURRENT STEPS: 2 
BEST MEMBER: 
 [-4.62857678e+00  1.02069395e+00  2.76731144e+00 -8.97253966e+00
  5.34788375e+00  4.96206154e-01  3 ......
BEST SCORE0.901099 

ES: 
CURRENT STEPS: 3 
BEST MEMBER: 
 [-4.81093381e+00  7.72393022e+00  2.16234740e+00 -5.71269837e+00
  4.82623451e+00  7.32845350e-01  4 ...